# COVID-19 cases in Kenya, preprocessing analysis

In [1]:
#load some key libraries
import pandas as pd
import datetime as dt
from datetime import date
import calendar
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
pcolor = "#FFFAFA"

In [2]:
! pwd

/Users/jmwanga/OneDrive - Kemri Wellcome Trust/Visualization/Cemia_Dash


In [4]:
#select_important_columns
#covid_data_original = pd.read_excel("Covid-19_cleaned_07-02-2022.xls", engine="xlrd")
covid_data_original = pd.read_table("/Users/jmwanga/OneDrive - Kemri Wellcome Trust/Visualization/COVID-19-Dashboard/covid_19_cleaned_01_11_2022_JM_processed.txt")

covid_data_original["county_where_the_case_was_diagonised"] =  covid_data_original["county_where_the_case_was_diagonised"].\
                                                            apply(lambda x : str(x).title())
covid_data_original["county_where_the_case_was_diagonised"] = covid_data_original["county_where_the_case_was_diagonised"].\
                    replace({"Murang'A": "Murang'a", "Kakamega ": "Kakamega", "Nakuru ": "Nakuru", "Kajiado ":"Kajiado"})


covid_data_original.head()

/tmp/ipykernel_68976/3211030212.py:3: DtypeWarning: Columns (2,3,15,16,17,25,26,28,29,35,36,37,38,39,40,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_data_original = pd.read_table("/Users/jmwanga/OneDrive - Kemri Wellcome Trust/Visualization/COVID-19-Dashboard/covid_19_cleaned_01_11_2022_JM_processed.txt")


,unique_id,case_id,contact_of_new,age_years,sex,occupation_sector_job_title_employer_,county_of_residence,county_where_the_case_was_diagonised,nationality,sub_county,...,co_morbidity,management,status_need,date_of_announcement_by_cs,type_of_testing,vaccinated,status,platform,history_of_contact_new,symptomatic_yes_no_new
0,Case-01,CASE 1,NaN,27.0,F,/UNEMPLOYED,KAJIADO,Kajiado,KENYAN,KAJIADO NORTH,...,NaN,NaN,NaN,13-Mar-20,PCR,#ERROR!,#ERROR!,1241,NO,YES
1,Case-02,CASE 2,NaN,48.0,F,/UNKNOWN,NAIROBI,Nairobi,KENYAN,STAREHE,...,NaN,NaN,NaN,NaN,PCR,#ERROR!,#ERROR!,NaN,NO,NO
2,Case-03,CASE 3,Case-01,27.0,M,/UNKNOWN,NAIROBI,Nairobi,KENYAN,KAJIADO NORTH,...,NaN,NaN,NaN,NaN,PCR,#ERROR!,#ERROR!,NaN,YES,NO
3,Case-04,CASE 4,NaN,31.0,F,/UNKNOWN,NAIROBI,Nairobi,KENYAN,LANGATA,...,NaN,NaN,NaN,NaN,PCR,#ERROR!,#ERROR!,NaN,NO,YES
4,Case-05,CASE 5,NaN,42.0,F,/UNKNOWN,NAIROBI,Nairobi,KENYAN,WESTLANDS,...,NaN,NaN,NaN,NaN,PCR,#ERROR!,#ERROR!,NaN,NO,YES


Visualizing lab_results column returns presence of wrong labels and names of hospitals. Creating a new colum fpor confirmed positives. This will determine the total cases

In [5]:
covid_data = covid_data_original[["unique_id","age_years", "sex","county_of_residence","county_where_the_case_was_diagonised",
                                    "status_need","status","lab_results","date_of_lab_confirmation","outcome_death_Discharge_still_in_hospital_"]]

covid_data.tail()

,unique_id,age_years,sex,county_of_residence,county_where_the_case_was_diagonised,status_need,status,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_
338469,Case-0338470,38.0,F,NAIROBI,Nairobi,NaN,C,POSITIVE,06/10/2022,NaN
338470,Case-0338471,27.0,F,NAIROBI,Nairobi,NaN,NaN,POSITIVE,06/10/2022,NaN
338471,Case-0338472,76.0,M,NAIROBI,Nairobi,NaN,NaN,POSITIVE,06/10/2022,NaN
338472,Case-0338473,34.0,M,NAKURU,Nakuru,NaN,NaN,POSITIVE,06/10/2022,NaN
338473,Case-0338474,19.0,F,NAKURU,Nakuru,NaN,NaN,POSITIVE,06/10/2022,NaN


In [6]:
#convert the date column to true dates
covid_data["date_of_lab_confirmation"] = pd.to_datetime(covid_data["date_of_lab_confirmation"], format="%d/%m/%Y", errors="coerce")
covid_data["lab_results"].replace({"POSITIVE":"Positive"},inplace=True)
covid_data["outcome_death_Discharge_still_in_hospital_"].replace({"DEAD":"Dead"},inplace = True) #the second file came with differenet name system for the fatalitites

/tmp/ipykernel_68976/2909419089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data["date_of_lab_confirmation"] = pd.to_datetime(covid_data["date_of_lab_confirmation"], format="%d/%m/%Y", errors="coerce")
/tmp/ipykernel_68976/2909419089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data["lab_results"].replace({"POSITIVE":"Positive"},inplace=True)
/tmp/ipykernel_68976/2909419089.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

## Plots based on lab_results and outcome from hospitals

In [7]:
covid_data_view = covid_data[[ "sex","age_years", "lab_results", "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" , "county_where_the_case_was_diagonised"]]

#there exists some cases with missing date of lab confirmation. Dropping them
covid_data_view=covid_data_view.dropna(subset=["date_of_lab_confirmation"])

covid_data_view.tail()

,sex,age_years,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_,county_where_the_case_was_diagonised
338469,F,38.0,Positive,2022-10-06,NaN,Nairobi
338470,F,27.0,Positive,2022-10-06,NaN,Nairobi
338471,M,76.0,Positive,2022-10-06,NaN,Nairobi
338472,M,34.0,Positive,2022-10-06,NaN,Nakuru
338473,F,19.0,Positive,2022-10-06,NaN,Nakuru


In [8]:
covid_data_view["Year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("Y")
covid_data_view["Month"] =  covid_data_view["date_of_lab_confirmation"].dt.month
covid_data_view["Week"] = covid_data_view["date_of_lab_confirmation"].dt.week
covid_data_view["Month"] =  covid_data_view["Month"].apply(lambda x: calendar.month_abbr[x])
covid_data_view["Month_year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("M")
covid_data_view["year_month"] = covid_data_view["Month"] + "-" +  covid_data_view["Year"].astype(str) 
covid_data_view.head()

/tmp/ipykernel_68976/3999872376.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  covid_data_view["Week"] = covid_data_view["date_of_lab_confirmation"].dt.week


,sex,age_years,lab_results,date_of_lab_confirmation,outcome_death_Discharge_still_in_hospital_,county_where_the_case_was_diagonised,Year,Month,Week,Month_year,year_month
0,F,27.0,Positive,2020-03-12,DISCHARGE,Kajiado,2020,Mar,11,2020-03,Mar-2020
1,F,48.0,Positive,2020-03-14,DISCHARGE,Nairobi,2020,Mar,11,2020-03,Mar-2020
2,M,27.0,Positive,2020-03-14,DISCHARGE,Nairobi,2020,Mar,11,2020-03,Mar-2020
3,F,31.0,Positive,2020-03-17,DISCHARGE,Nairobi,2020,Mar,12,2020-03,Mar-2020
6,M,27.0,Positive,2020-03-17,DISCHARGE,Nairobi,2020,Mar,12,2020-03,Mar-2020


In [9]:
Monthly_cases = covid_data_view[["Month_year", "year_month", "outcome_death_Discharge_still_in_hospital_" ,"lab_results"]].\
    groupby(["Month_year", "year_month"])["Month_year"].count().to_frame(name = "COVID19_Cases").reset_index()
Monthly_cases.rename(columns = {"COVID19_Cases": "Reported_cases"}, inplace=True)

death_cases = covid_data_view[covid_data_view["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
monthly_death_cases = death_cases[["outcome_death_Discharge_still_in_hospital_","Month_year", "year_month"]].\
    groupby(["Month_year", "year_month"]).count().reset_index()
monthly_death_cases.rename(columns = {"outcome_death_Discharge_still_in_hospital_" : "Death_cases"}, inplace=True)

covid_monthly_data = pd.merge(Monthly_cases, monthly_death_cases, on = "year_month", how = "outer")
covid_monthly_data =  covid_monthly_data.drop("Month_year_x", axis=1)
covid_monthly_data =  covid_monthly_data.rename(columns={"Month_year_y" : "Month_year"})
covid_monthly_data["Month_year"]= covid_monthly_data["Month_year"].dt.strftime("%Y-%m")
covid_monthly_data.to_csv("../COVID-19-Dashboard2/data/covid_monthly_data.csv")

## covid_monthly_data.to_csv("../ /covid_monthly_data.csv")


Daily Cases

In [10]:
#summary for daily reported dcases
daily_cases =  covid_data_view[["date_of_lab_confirmation", "lab_results"]].groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].\
    count().to_frame(name = "Reported_Cases").reset_index()
daily_cases.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)
ily_deaths = daily_deaths[[ "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" ]].\
    groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].count().to_frame(name="death_cases").reset_index()
daily_deaths.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)

#summary for dicharged
daily_discharges = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Discharge"]
daily_discharges = daily_discharges[[ "date_of_lab_confirmation", "outcome_death_Discharge_still_in_hospital_" ]].\
    groupby("date_of_lab_confirmation")["date_of_lab_confirmation"].count().to_frame(name="Reported_discharges").reset_index()
daily_discharges.rename(columns = {"date_of_lab_confirmation": "Date"}, inplace=True)

covid_daily_data = pd.merge(pd.merge(daily_cases, daily_deaths, how = "outer"), daily_discharges, how="outer")

covid_daily_data["death_cases"] = covid_daily_data["death_cases"].fillna(0)
covid_daily_data["Reported_discharges"] = covid_daily_data["Reported_discharges"].fillna(0)

covid_daily_data = covid_daily_data.astype({"Reported_Cases":int, "death_cases": int, "Reported_discharges":int })
covid_daily_data["Cum_Cases"] = covid_daily_data["Reported_Cases"].cumsum()
covid_daily_data["Cum_Deaths"] = covid_daily_data["death_cases"].cumsum()
#covid_daily_data.to_csv("../COVID-19-Dashboard2/data/covid_daily_data.csv")
covid_daily_data.head()#daily_cases.tail()

#summary for dail deaths
daily_deaths = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Dead"]


#7 day windo average cases
window_size = 7
moving_average_cases = []
data = covid_daily_data["Reported_Cases"]
for i in range(len(data)):
    if i + window_size < len(data):
        moving_average_cases.append(round(np.mean(data[i:i+window_size]),1))
    else:
        moving_average_cases.append(np.mean(data[i:len(data)]))

moving_average_deaths = []
data = covid_daily_data["death_cases"]
for i in range(len(data)):
    if i + window_size < len(data):
        moving_average_deaths.append(round(np.mean(data[i:i+window_size]),1))
    else:
        moving_average_deaths.append(np.mean(data[i:len(data)]))


covid_daily_data["moving_average_cases"]  = moving_average_cases
covid_daily_data["moving_average_deaths"] = moving_average_deaths
covid_daily_data.to_csv("../COVID-19-Dashboard2/data/covid_daily_data.csv")
covid_daily_data.head()

,Date,Reported_Cases,death_cases,Reported_discharges,Cum_Cases,Cum_Deaths,moving_average_cases,moving_average_deaths
0,2020-03-12,1,0,0,1,0,2.3,0.0
1,2020-03-14,2,0,0,3,0,3.4,0.1
2,2020-03-17,2,0,0,5,0,3.7,0.1
3,2020-03-19,3,0,0,8,0,3.7,0.1
4,2020-03-20,1,0,0,9,0,3.4,0.3


Weekly Cases

In [ ]:
#export data

covid_data.to_csv("covid_data_processed.csv")

## Visualization Based on County where case was diagnoised


In [12]:
#covid_data.rename(columns = {"county_where_the_case_was_diagonised" :"County"}, inplace=True)
county_death = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Dead"]
#county_death.rename(columns = {"county_where_the_case_was_diagonised" :"County"}, inplace=True)

county_death_counts = county_death[["county_where_the_case_was_diagonised", "outcome_death_Discharge_still_in_hospital_"]].\
                        groupby("county_where_the_case_was_diagonised").count().reset_index()
                        
county_death_counts.rename(columns={"outcome_death_Discharge_still_in_hospital_": "Death_cases"}, inplace=True)
#county_death_counts

discharged = covid_data[covid_data["outcome_death_Discharge_still_in_hospital_"] == "Discharge"]
discharged = discharged[["county_where_the_case_was_diagonised", "outcome_death_Discharge_still_in_hospital_"]].\
                    groupby("county_where_the_case_was_diagonised").count().reset_index()
discharged.rename(columns={"outcome_death_Discharge_still_in_hospital_": "Discharged"}, inplace=True)
#discharged

#covid_data["county_of_residence"] = covid_data["county_where_the_case_was_diagonised"].str.strip().str.title()#.unique() #Needs clean-up for naming consistency
county_prevalence = covid_data.groupby("county_where_the_case_was_diagonised")["county_where_the_case_was_diagonised"].count().\
                to_frame(name = "cases").reset_index()
county_prevalence = county_prevalence.sort_values("cases", ascending=False)
#county_prevalence

county_cases = pd.merge(pd.merge(county_prevalence, county_death_counts, on="county_where_the_case_was_diagonised"),\
                    discharged,how="left", on="county_where_the_case_was_diagonised")

county_cases["Discharged"] = county_cases["Discharged"].fillna(0)
county_cases = county_cases.astype({"cases":int, "Death_cases": int, "Discharged":int })

county_cases.rename(columns={"county_where_the_case_was_diagonised": "County"}, inplace=True)

#include sequenced, vaccination, status


county_cases.to_csv("../COVID-19-Dashboard2/data/cases_per_county.csv") #check merged with sequence information

county_cases.head()

,County,cases,Death_cases,Discharged
0,Nairobi,140014,1558,0
1,Kiambu,21223,552,0
2,Mombasa,18217,337,0
3,Nakuru,17613,541,0
4,Uasin Gishu,10860,243,0


## Vaccination information

In [13]:
#data from ministry of health 

County = ["Baringo","Bomet","Bungoma","Busia","Elgeyo Marakwet","Embu","Garissa","Homa Bay","Isiolo","Kajiado","Kakamega","Kericho","Kiambu",\
    "Kilifi","Kirinyaga","Kisii","Kisumu","Kitui","Kwale","Laikipia","Lamu","Machakos","Makueni","Mandera","Marsabit","Meru","Migori","Mombasa",\
            "Murang'a","Nairobi","Nakuru","Nandi","Narok","Nyamira","Nyandarua","Nyeri","Samburu","Siaya","Taita Taveta","Tana River","Tharaka Nithi",\
                "Trans Nzoia","Turkana","Uasin Gishu","Vihiga","Wajir","West Pokot"]
Population_vaccinated = [84904,76594,279522,161039,66826,133100,54335,233810,17548,231369,434677,166407,594614,136859,157439,258899,293662,138916,\
                                78222,122869,19682,323810,192008,35563,22963,282518,220721,275349,253870,1597784,518376,132910,84855,141630,124173,\
                                    273192,31802,237793,82965,16606,85121,191032,102890,242715,113992,37371,41559]
                    
Proportion_vaccinated = [25.1,16.4,33.1,34.6,27.4,33.6,13.6,38.4,13.2,34.9,44.2,32.7,36.6,17.4,38.1,37.1,44.8,22.2,17.3,40.1,23.6,35.8,32.9,10.4,10.5,\
                        29.8,39.6,34.2,37.6,52.3,40.7,27,15.5,41.5,33,53.6,21.8,44.4,38.9,11,34.5,36.9,21.9,35,34.6,11.1,14.6]

#pd.DataFrame(zip(County,Population_vaccinated,Proportion_vaccinated))
vaccination_per_county = pd.DataFrame(zip(County, Population_vaccinated, Proportion_vaccinated), columns = ["County","Population_vaccinated", "Proportion_vaccinated" ])

#vaccination_per_county.to_csv("county_vaccination.csv")
vaccination_per_county.sort_values("Proportion_vaccinated", ascending=True).to_csv("../COVID-19-Dashboard2/data/county_vaccination.csv")


# creating regions from county

In [ ]:
county_cases.head()

,County,cases,Death_cases,Discharged
0,Nairobi,130228,1542,3561
1,Kiambu,20127,545,263
2,Mombasa,17887,337,1530
3,Nakuru,16947,531,66
4,Uasin Gishu,10507,241,111


In [14]:

regions = dict(
    {"Coast" : ["Mombasa", "Kwale", "Kilifi", "Tana River", "Taita Taveta", "Lamu"],
    "North Eastern" : [  "Garissa", "Wajir", "Mandera"],
    "Eastern" : ["Marsabit", "Isiolo", "Meru", "Tharaka Nithi", "Embu", "Kitui", "Machakos", "Makueni"],
    "Central" : [ "Nyandarua", "Murang'a", "Kirinyaga", "Nyeri", "Kiambu"],
    "Rift Valley" : ["Nakuru", "Narok", "Kajiado", "Kericho", "Bomet", "Turkana", "West Pokot", "Samburu","Trans Nzoia", "Uasin Gishu", "Elgeyo Marakwet", "Nandi", "Baringo", "Laikipia"],
    "Western": ["Kakamega", "Vihiga", "Bungoma", "Busia"],
    "Nyanza": [ "Siaya", "Kisumu", "Homa Bay", "Migori", "Kisii", "Nyamira"],
    "Nairobi": ["Nairobi"]
    }) 

#print(regions.values())
province = []
for name in county_cases["County"]:
    for (k,v) in regions.items():
        for i in v:
            if i == name:
                province.append(k)
        

county_cases["province"] =  province

county_cases.head()

,County,cases,Death_cases,Discharged,province
0,Nairobi,140014,1558,0,Nairobi
1,Kiambu,21223,552,0,Central
2,Mombasa,18217,337,0,Coast
3,Nakuru,17613,541,0,Rift Valley
4,Uasin Gishu,10860,243,0,Rift Valley


In [15]:
import geopandas as gpd
#import geoplot as gplt
#import geoplot.crs as gcrs
#import imageio
import matplotlib.pyplot as plt
#import mpclassift as pc


province_cases = county_cases.groupby(["province"])["cases","Death_cases", "Discharged"].sum().reset_index()

kenya_province = gpd.read_file("Admin2/KEN_admin2_2002_DEPHA.shp")
kenya_province["PROVINCE"] =  kenya_province["PROVINCE"].str.replace("\r\n", "").str.title()

province_cases = pd.merge(kenya_province, province_cases, how="left", left_on=["PROVINCE"], right_on=["province"])

province_cases.to_csv("../COVID-19-Dashboard2/data/provice_level_cases.csv")


province_cases

/tmp/ipykernel_68976/1143719482.py:9: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



DriverError: Admin2/KEN_admin2_2002_DEPHA.shp: No such file or directory

## Visualizing death cases

Prepare dataset to be used for county level comparison - time series

In [16]:
county_daily_data = covid_data_view[covid_data_view["lab_results"] == "Positive"]

county_daily_data = county_daily_data.rename(columns = { "outcome_death_Discharge_still_in_hospital_": "outcome", 
                            "county_where_the_case_was_diagonised" : "county"})

county_daily_data.to_csv("../COVID-19-Dashboard2/data/county_daily_data.csv")
county_daily_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332427 entries, 0 to 338473
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   sex                       332415 non-null  object        
 1   age_years                 331375 non-null  object        
 2   lab_results               332427 non-null  object        
 3   date_of_lab_confirmation  332427 non-null  datetime64[ns]
 4   outcome                   12411 non-null   object        
 5   county                    332427 non-null  object        
 6   Year                      332427 non-null  period[A-DEC] 
 7   Month                     332427 non-null  object        
 8   Week                      332427 non-null  int64         
 9   Month_year                332427 non-null  period[M]     
 10  year_month                332427 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7), period[A-DEC](1), peri

## Age based analysis with gender

In [ ]:
age_gender = county_daily_data.reset_index()[["sex", "age_years", "lab_results"]]
age_gender =  age_gender[(age_gender["age_years"].notna()) & (age_gender["age_years"] != "M")]  #take rows that are not na
age_gender["age_years"] =  age_gender["age_years"].astype(int)
age_gender = age_gender[(age_gender["age_years"] > 0) & age_gender["age_years"] < 100]
bins  = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,100 ]
labels = ["0-5", "6-10", "11-15","16-20","21-25", "25-30", "31-35", "36-40", "41-45","46-50","51-55","56-60", "61-65","66-70","71-75",
            "76-80","81-85","86-90","90+"]
age_gender["age_groups"] = pd.cut(age_gender.age_years, bins, labels=labels, include_lowest=True)

age_gender["sex"] = age_gender["sex"].replace({"m":"M", "f":"F"})
age_gender = age_gender.rename(columns={"sex":"Gender"})
age_gender = age_gender[["Gender","age_years","age_groups"]].groupby(["age_groups","Gender"]).count().reset_index()
age_gender = pd.pivot_table(age_gender, values="age_years", index = ["age_groups"],columns = ["Gender"])
age_gender = age_gender.rename(columns = {"F":"Female","M":"Male"})
age_gender.to_csv("age_gender_cases_data.csv")


### Age gender death cases

In [ ]:
county_daily_data_selected = county_daily_data[county_daily_data["county"].isin(["Nairobi", "Mombasa"])]
county_daily_data_selected = county_daily_data_selected[[ "county", "lab_results", "date_of_lab_confirmation"]].\
                                groupby(["county", "date_of_lab_confirmation"]).count().reset_index()

fig = px.line(county_daily_data_selected, x="date_of_lab_confirmation" , y = "lab_results" , color = "county")
fig.update_traces(mode = 'lines')
fig.update_layout()
fig.show()

In [ ]:
county_daily_data["date_of_lab_confirmation"] = pd.to_datetime(county_daily_data["date_of_lab_confirmation"])
county_daily_data["Date"] = county_daily_data["date_of_lab_confirmation"]#.dt.date
county_daily_data.set_index("Date", inplace = True)


## Seroprecalence Data preprocessing

In [ ]:
sero_data = pd.read_excel("KWTRP_serosurveillance_data_Dashboard_09Sep2022.xlsx") #requires openpyxl
sero_data["Period"] = sero_data[["Month(s)", "Year"]].astype(str).agg(" ".join, axis=1)
daily_cases = pd.read_csv("covid_daily_data.csv")

## Genomics Data Preprocessing
Sequence metadata were downloaded from GISAID Databases and merged together

In [ ]:
seq_metadata["Region"] = seq_metadata["Location"].str.split("/",2,expand=True)[2] #select the third colum representing counties

seq_metadata["Region"] = seq_metadata.Region.fillna("unknown")
seq_metadata["Region"] = seq_metadata["Region"].str.replace(" ","")

seq_metadata[seq_metadata.Region == "unknown"]

unknown_regions = []
for name in seq_metadata["Region"]:
    if name not in County:
        unknown_regions.append(name)

unknown_regions = set(unknown_regions) #get unique values

NameError: name 'seq_metadata' is not defined

In [ ]:

for name in seq_metadata["Region"]:
    if name.startswith("Bomet"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Bomet"})
    elif name.startswith("Dadaab"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Garissa"})
    elif name.startswith("Homa"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Homa Bay"})
    elif name.startswith("Kerugoya"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Kirinyaga"})
    elif name.startswith(("Kuria")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Migori"})
    elif name.startswith(("Machakos","MACHA")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Machakos"})
    elif name.startswith("Makindu"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Makueni"})
    elif name.startswith(("Malaba","Buisa")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Busia"})
    elif name.startswith("Meru"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Meru"})
    elif name.startswith("Moyale"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Marsabit"})
    elif name.startswith(("EMT","Elgeyo")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Elgeyo Marakwet"})
    elif name.startswith("Trans"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Trans Nzoia"})
    elif name.startswith("Keri"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Kericho"})
    elif name.startswith(("Westp","Pokot")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"West Pokot"})
    elif name.startswith(("Uasin","Kapseret","Eldoret")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Uasin Gishu"})
    elif name.startswith("Tharaka"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Tharaka Nithi"})
    elif name.startswith("Tana"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Tana River"})
    elif name.startswith("Taita"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Taita Taveta"})
    elif name.startswith("Nairob"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Nairobi"})
    elif name.startswith(("Thika","KIAMBU")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Kiambu"})
    elif name.startswith(("Wakiso","NorthKivu","Mukono","Kyadondo","Kampala")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"unknown"})
    elif name.startswith(("Nyando","Kisumu")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Kisumu"})
    elif name.startswith(("Nanyuki","Laiki")):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Laikipia"})
    elif name.startswith("Naivasha"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Nakuru"})
    elif name.startswith("Muran"):
        seq_metadata["Region"] =  seq_metadata["Region"].replace({name:"Murang'a"})


In [ ]:
daily_seq_data = seq_metadata[["collection_date", "Accession ID"]].groupby("collection_date").count().reset_index()
daily_seq_data = daily_seq_data.rename(columns={"Accession ID":"sequences"})
daily_seq_data["cum_sequences"] = daily_seq_data["sequences"].cumsum()

daily_seq_data.head()

,collection_date,sequences,cum_sequences
0,2020-03-01,9,9
1,2020-03-06,1,10
2,2020-03-14,1,11
3,2020-03-17,2,13
4,2020-03-20,1,14


In [ ]:
seq_by_region.to_csv("sequence_metadata_region.tsv", sep="\t")
daily_seq_data.to_csv("sequence_metadata.tsv", sep="\t")


## Trends in sample sequencing

## Location

## Genomics

In [ ]:
! pwd

/Users/jmwanga/OneDrive - Kemri Wellcome Trust/Visualization/COVID-19-Dashboard


In [ ]:
import pandas as pd
genomics_data = pd.read_table("../Genomics/genomics_data2.txt")
variants = genomics_data.copy()

In [ ]:
len(genomics_data)

11591

In [ ]:
variants["variant"] =  variants["variant"].str.replace(" ","")
variants["variant"] =  variants["variant"].str.replace("\n","")
variants["variant"] =  variants["variant"].str.strip()

KeyError: 'variant'

In [ ]:
#var_name = []
for lineage in variants["variant"]:
    #print(lineage)
    if lineage.startswith("AY"):
       variants["variant"] = variants["variant"].replace({lineage: "Delta"})
    elif lineage.startswith(("Q", "B.1.1.7", "B.1.617.2")):
        variants["variant"] = variants["variant"].replace({lineage: "Alpha"})
    elif lineage.startswith("B.1.351"):
        variants["variant"] = variants["variant"].replace({lineage: "Beta"})
    elif lineage.startswith("P.1"):
        variants["variant"] = variants["variant"].replace({lineage: "Gamma"})
    elif lineage.startswith(("B.1.1.529", "BA","BA.1.1")):
        variants["variant"] = variants["variant"].replace({lineage: "Omicron"})
    elif lineage.startswith(("B.1.427", "B.1.429")):
        variants["variant"] = variants["variant"].replace({lineage: "Epilson"})
    elif lineage.startswith(("B.1.525")):
        variants["variant"] = variants["variant"].replace({lineage: "Eta"})
    elif lineage.startswith(("B.1.526")):
        variants["variant"] = variants["variant"].replace({lineage: "Iota"})
    elif lineage.startswith(("B.1.617.1")):
        variants["variant"] = variants["variant"].replace({lineage: "Kappa"})

    

In [ ]:
genomics_data["date"] = pd.to_datetime(genomics_data["date"],format="%d/%m/%Y",infer_datetime_format=True)
genomics_data["date_submitted"] = pd.to_datetime(genomics_data["date_submitted"],format="%d/%m/%Y",infer_datetime_format=True)

In [ ]:
variants.head(n=2)

,Unnamed: 0,strain,virus,gisaid_epi_isl,genbank_accession,date,region,country,division,location,...,pangolin_lineage,GISAID_clade,originating_lab,submitting_lab,authors,url,title,paper_url,date_submitted,purpose_of_sequencing
0,406,hCoV-19/Kenya/C11204/2020,betacoronavirus,EPI_ISL_806622,?,01/06/2020,Africa,Kenya,NaN,NaN,...,B.1,G,KEMRI-Wellcome Trust Research Programme/KEMRI-...,KEMRI-Wellcome Trust Research Programme/KEMRI-...,Githinji et al,https://www.gisaid.org/,?,?,13/01/2021,?
1,208,hCoV-19/Kenya/NIC_488/2020,betacoronavirus,EPI_ISL_457839,?,01/03/2020,Africa,Kenya,NaN,NaN,...,B.1,GH,National Public Health Laboratory,KEMRI-Wellcome Trust Research Programme/KEMRI-...,Githinji G. et al 2020,https://www.gisaid.org/,?,?,02/06/2020,?


## Participating Labs

In [ ]:
genomics_data["submitting_lab"] = genomics_data["submitting_lab"].str.strip()
genomics_data["submitting_lab"].value_counts()

KEMRI-Wellcome Trust Research Programme,Kilifi         7472
International Livestock Research Institute             2631
USAMRU-Kenya                                           1144
KEMRI VHF/FLU LAB                                       141
DLSP KEMRI/CDC Lab                                      102
ARI/CVR, KEMRI                                           63
KEMRI & ICGEB                                            12
Pan African University Institute for Basic Sciences       9
Africa Genomics Centre and Consultancy                    5
CERI-UKZN                                                 5
Gujarat Biotechnology Research Centre                     3
Centre For Biotechnology Research and Development         2
Arizona State University                                  2
Name: submitting_lab, dtype: int64

In [ ]:

genomics_data["submitting_lab"] = genomics_data["submitting_lab"].replace(["KEMRI-Wellcome Trust Research Programme/KEMRI-CGMR-C Kilifi",
                                "KEMRI-Wellcome Trust Research Programme", "Epidemiology and Demography,  KEMRI-Wellcome Trust Research Programme",
                                "Epidemiology and Demography, KEMRI-WELLCOME TRUST","Epidemiology and Demography, KEMRI-WELLCOME TRUST",
                                "Epidemiology and Demography, KEMRI-Wellcome Trust Research Programme",
                                ], "KEMRI-Wellcome Trust Research Programme,Kilifi")


genomics_data["submitting_lab"] = genomics_data["submitting_lab"].replace(["Diagnostics and Laboratory Systems Program Lab, KEMRI-CGHR",
                                                "DP LSKEMRI-CDC","DLSP KEMRI/CDC Lab","DLSP KEMRI/CDC NAIROBI Lab","CDC-DLSP-Kisumu"], "DLSP KEMRI/CDC Lab")

genomics_data["submitting_lab"] = genomics_data["submitting_lab"].replace(["United States Army Medical Research Directorate, Kenya",
                                                "USAMRD-A, Basic Science Laboratory"],"USAMRU-Kenya")

genomics_data["submitting_lab"] = genomics_data["submitting_lab"].replace(["CERI, Centre for Epidemic Response and Innovation, Stellenbosch University and KRISP, KZN Research Innovation and Sequencing Platform, UKZN."],"CERI-UKZN")

genomics_data["submitting_lab"] = genomics_data["submitting_lab"].replace(["THE AFRICA GENOMICS CENTRE AND CONSULTANCY LIMITED",
                                                        "THE AFRICA GENOMICS CENTRE AND CONSULTANCY"],"Africa Genomics Centre and Consultancy")

genomics_data["submitting_lab"] =  genomics_data["submitting_lab"].replace(["ARI lab, Center for Virus Research, Kenya Medical Research Institute"],"ARI/CVR, KEMRI")


genomics_data["submitting_lab"] = genomics_data["submitting_lab"].replace("Animal and Human Health Unit, International Livestock Research Institute",
                                        "International Livestock Research Institute")


genomics_data["submitting_lab"] = genomics_data["submitting_lab"].replace("Biochemistry, Pan African University Institute for Basic Sciences","Pan African University Institute for Basic Sciences")

In [ ]:
labs = genomics_data[["submitting_lab","date", "date_submitted"]]
labs["date_submitted"] = pd.to_datetime(labs['date_submitted'],format = "%d/%m%Y")
labs["sampled_date"] = pd.to_datetime(labs["date"], format = "%d/%m/%Y")
labs["submitted_month"] = labs["date_submitted"].dt.to_period("M")
labs["sampled_month"] = labs["sampled_date"].dt.to_period("M")

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_35365/884595082.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_35365/884595082.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/0x/t7x8m9311b1gnnb39j773h8c3q8zn2/T/ipykernel_35365/884595082.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [ ]:

labs_data = labs.groupby(["sampled_month","submitting_lab"])["submitting_lab"].count()#.r
labs_data = pd.DataFrame(labs_data)
labs_data = labs_data.rename(columns = {"submitting_lab":"Counts"}).reset_index()
labs_data["percentage"] = labs_data.groupby("sampled_month")["Counts"].apply(lambda x: x/x.sum())
summed = labs_data.groupby(["sampled_month"])["Counts"].sum().reset_index()
labs_data = pd.merge(labs_data, summed, on="sampled_month")
labs_data.rename(columns ={"Counts_x":"Counts","Counts_y":"Total_submitted"}, inplace=True)
labs_data.to_csv("./submiting_labs_monthly.csv")


In [ ]:
labs_data.head()

,sampled_month,submitting_lab,Counts,percentage,Total_submitted
0,2020-03,Centre For Biotechnology Research and Development,1,0.032258,31
1,2020-03,"KEMRI-Wellcome Trust Research Programme,Kilifi",30,0.967742,31
2,2020-04,KEMRI VHF/FLU LAB,1,0.008475,118
3,2020-04,"KEMRI-Wellcome Trust Research Programme,Kilifi",117,0.991525,118
4,2020-05,Centre For Biotechnology Research and Development,1,0.004902,204


In [ ]:
labs = pd.read_csv("./submiting_labs_monthly.csv")
labs.head()

,Unnamed: 0,sampled_month,submitting_lab,Counts,percentage,Total_submitted
0,0,2020-03,Centre For Biotechnology Research and Development,1,3.225806,31
1,1,2020-03,"KEMRI-Wellcome Trust Research Programme,Kilifi",30,96.774194,31
2,2,2020-04,KEMRI VHF/FLU LAB,1,0.847458,118
3,3,2020-04,"KEMRI-Wellcome Trust Research Programme,Kilifi",117,99.152542,118
4,4,2020-05,Centre For Biotechnology Research and Development,1,0.490196,204


## Variants

In [ ]:
variant_data = variants[variants["variant"].isin(["Alpha","Beta", "Delta","Omicron","Gamma"])]

variant_data = variant_data[["strain","date","variant","submitting_lab","date_submitted"]]

variant_data["date"] = pd.to_datetime(variant_data["date"],format="%d/%m/%Y",infer_datetime_format=True)
variant_data["Year"] = variant_data["date"].dt.to_period("Y")
#variant_data["Month"] = variant_data["date"].dt.to_period("M")
#covid_data_view["Year"] = covid_data_view["date_of_lab_confirmation"].dt.to_period("Y")
#covid_data_view["Month"] =  covid_data_view["date_of_lab_confirmation"].dt.month

In [ ]:
variant_month = variant_data[['strain',"Month","variant"]].groupby(["Month","variant"]).count().reset_index()
variant_month["Percent"] = variant_month.groupby("Month")["strain"].apply(lambda x: x *100/x.sum())
variant_month["Month"] = variant_month["Month"].astype(str)

,Month,variant,strain
0,2020-09,Alpha,9
1,2020-09,Delta,1
2,2020-10,Alpha,1
3,2020-11,Delta,2
4,2020-12,Alpha,11


## Lineages Data

In [ ]:


lineage_data = pd.read_table("lineages_per_month.tsv")
lineage_data["months_period"] = pd.to_datetime(lineage_data["month"]).dt.to_period("M")

lineage_data.head()

,month,variant_type,lineage_count,total,proportion,months_period
0,2020-03-01,Non-VOC/VOI,13,13,1.000000,2020-03
1,2020-04-01,Non-VOC/VOI,96,96,1.000000,2020-04
2,2020-05-01,Non-VOC/VOI,141,141,1.000000,2020-05
3,2020-06-01,Non-VOC/VOI,121,122,0.991803,2020-06
4,2020-06-01,Omicron,1,122,0.008197,2020-06


In [ ]:
variants_data =  lineage_data.groupby(["months_period","variant_type"])["lineage_count"].sum().reset_index()

variants_data["percentage"] = variants_data.groupby("months_period")["lineage_count"].apply(lambda x: x*100/x.sum())
variants_data["months_period"] = variants_data["months_period"].astype(str)
variants_data.head()
variants_data = variants_data[:-1]
variants_data.to_csv("variant_month_data.txt", sep = "\t")

In [ ]:
variants_data = variants_data[:-1]

,months_period,variant_type,lineage_count,percentage
0,2020-03,Non-VOC/VOI,13,100.000000
1,2020-04,Non-VOC/VOI,96,100.000000
2,2020-05,Non-VOC/VOI,141,100.000000
3,2020-06,Non-VOC/VOI,121,99.180328
4,2020-06,Omicron,1,0.819672
...,...,...,...,...
76,2022-03,Omicron,11,100.000000
77,2022-04,Omicron,13,100.000000
78,2022-05,Omicron,118,100.000000
79,2022-06,Non-VOC/VOI,13,2.292769


In [ ]:
px.bar(variants_data, x = "months_period", y = "percentage", color="variant_type")

In [ ]:
genome_plot = make_subplots(rows = 2,cols=1, vertical_spacing = 0.2,shared_xaxes=True)
genome_plot.add_trace(go.Scatter( x = daily_deaths.Date,  y = daily_deaths.death_cases,mode='none',
                            fill = 'tozeroy', marker = dict(color = "#A3A3A3")), row=1,col=1)

#genome_plot.add_trace(px.bar(variant_data, x = "months_period", y = "percentage", color="variant_type"),row=2,col=1)#, color=variants_data["variant_type"]))

genome_plot.show()



In [ ]:
 #All exports
#county_cases.to_csv("cases_per_county.csv")

## Putting this in a class instance.

In [ ]:
class Variants:
    def __init__(self, variant, perc):
        self.variant = variant
        self.perc = perc
    
v = Variants("Alpha", 30)

v.variant


'Alpha'

In [ ]:
variant_data = pd.read_table("variant_month_data.txt")
submitting_labs = pd.read_table("submitting_labs.txt")

class Variants:
    
    #percentage = "percentage"
    def __init__(self, percentage):
        self.percentage = percentage

    def variants_prevalence(self):
        variant_plot = px.bar(variant_data, x = "months_period", y = self.percentage, color="variant_type")
        variant_plot.update_layout(paper_bgcolor = pcolor,plot_bgcolor = pcolor)
        variant_plot.update_xaxes(title = "Time")
        variant_plot.update_yaxes(title = "Proportion")
        return variant_plot


    def labs(self):
        submitting_labs_plot = px.bar(submitting_labs, x = "submitted_month", y = self.percentage, color="submitting_lab")
        return submitting_labs_plot

In [ ]:
ps = Variants("percentage")

ps.variants_prevalence()

## Creating Interactive Maps

In [ ]:
import plotly.express as px
import geopandas as gpd


In [ ]:
def seven_day_average(data,column):
    window_size = 7
    moving_average = []
    data_col = data[column]
    for i in range(len(data)):
        if i + window_size < len(data_col):
            moving_average.append(round(np.mean(data_col[i:i+window_size]),1))
        else:
            moving_average.append(np.mean(data_col[i:len(data_col)])) 
    data["moving_average"] = moving_average
    return data

data_nai = seven_day_average(data,"lab_results" )


,county,date_of_lab_confirmation,lab_results,moving_average
0,Nairobi,2020-03-23,1,1.0
1,Nairobi,2020-03-27,1,1.0
2,Nairobi,2020-03-29,1,1.0
3,Nairobi,2020-04-01,1,1.0
4,Nairobi,2020-04-02,1,1.0
...,...,...,...,...
461,Nairobi,2022-01-18,2,1.2
462,Nairobi,2022-01-21,1,1.0
463,Nairobi,2022-01-22,1,1.0
464,Nairobi,2022-01-23,1,1.0


In [ ]:
round(data_nai.moving_average)

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
461    1.0
462    1.0
463    1.0
464    1.0
465    1.0
Name: moving_average, Length: 466, dtype: float64

In [ ]:
px.line(data_nai, x = "date_of_lab_confirmation", y = "moving_average")

In [ ]:
import PyPDF2

In [ ]:
file_path = "/Users/jmwanga/OneDrive - Kemri Wellcome Trust/Monkey_Pox/Papers/monkeypox-virus-whole-genome-sequencing-using-comb-ccc7sszn (1).pdf"

outfile = "/Users/jmwanga/OneDrive - Kemri Wellcome Trust/Monkey_Pox/Papers/files.doc"
n = 2
while n < 11:
    with open(file_path, mode="rb") as f:
        with open(outfile, "a") as of:
            reader = PyPDF2.PdfFileReader(f)
            page = reader.getPage(n)
            of.write(page.extractText())
            n+=1

In [ ]:
vaccination_updates = pd.read_csv("../Vaccination_Data/vaccination_metadata_october.csv", index_col="Group")
vaccination_updates.drop("Unnamed: 0", inplace=True, axis=1)
vaccination_updates.columns = pd.to_datetime(vaccination_updates.columns, format="%d/%m/%Y")


In [ ]:
last_updated = vaccination_updates.columns[-1].strftime(format="%d-%B-%Y")

vaccination_updates[vaccination_updates.index == "Total doses administered adult population (18 years and above)"].iloc[0][-1]

'18,162,557'

In [ ]:
last_7_days_df = vaccination_updates.iloc[:,-7:]
last_7_days_df =  last_7_days_df.loc[["Total doses administered","Fully vaccinated adult population (18 years and above)"]]
total_doses_7_days = int(last_7_days_df.loc[["Total doses administered"]].iloc[-1].iat[-1].replace(",","")) - int(last_7_days_df.loc[["Total doses administered"]].iloc[0].iat[0].replace(",",""))

fully_vaccinated_last_7 = int(last_7_days_df.loc[["Fully vaccinated adult population (18 years and above)"]].iloc[-1].iat[-1].replace(",","")) - int(last_7_days_df.loc[["Fully vaccinated adult population (18 years and above)"]].iloc[0].iat[0].replace(",",""))


51272

In [2]:
import pandas as pd

In [77]:
data = pd.read_csv("../data/covid_daily_data.csv")
daily_updates =  pd.read_excel("../data/daily_updates_metadata.xlsx")
previous_data = data[["Date","Reported_Cases","death_cases"]]

recent_data = daily_updates[["Date", "new_cases_last_24_hrs","recorded_deaths_last_24_hrs"]]
recent_data = recent_data.rename(columns={"new_cases_last_24_hrs":"Reported_Cases","recorded_deaths_last_24_hrs":"death_cases" })
recent_data["Date"] = pd.to_datetime(recent_data["Date"], format = "%Y/%m/%d")

updated_data = pd.concat([previous_data, recent_data],axis=0)
updated_data["Date"] = pd.to_datetime(updated_data["Date"])
updated_data.set_index("Date", inplace = True)
updated_data["Cum_Cases"] = updated_data["Reported_Cases"].cumsum()
updated_data["Cum_Deaths"] = updated_data["death_cases"].cumsum()
updated_data.head()


,Reported_Cases,death_cases,Cum_Cases,Cum_Deaths
Date,,,,
2020-03-12,1,0,1,0
2020-03-14,2,0,3,0
2020-03-17,2,0,5,0
2020-03-19,3,0,8,0
2020-03-20,1,0,9,0


In [78]:
def seven_day_average(data,column,new_name):
    window_size = 7
    moving_average = []
    data_col = data[column]
    for i in range(len(data)):
        if i + window_size < len(data_col):
            moving_average.append(round(np.mean(data_col[i:i+window_size]),1))
        else:
            moving_average.append(np.mean(data_col[i:len(data_col)])) 
    data[new_name] = moving_average
    return data

In [79]:
seven_day_average(updated_data,"Reported_Cases","moving_average_cases")
seven_day_average(updated_data,"death_cases","moving_average_deaths")

,Reported_Cases,death_cases,Cum_Cases,Cum_Deaths,moving_average_cases,moving_average_deaths
Date,,,,,,
2020-03-12,1,0,1,0,2.30,0.0
2020-03-14,2,0,3,0,3.40,0.1
2020-03-17,2,0,5,0,3.70,0.1
2020-03-19,3,0,8,0,3.70,0.1
2020-03-20,1,0,9,0,3.40,0.3
...,...,...,...,...,...,...
2023-01-22,3,0,342133,5687,5.60,0.0
2023-01-23,1,0,342134,5687,6.25,0.0
2023-01-24,6,0,342140,5687,8.00,0.0


In [81]:
updated_data.to_csv("../data/covid_daily_data.csv")

In [23]:

#7 day windo average cases
window_size = 7
moving_average_cases = []
data = covid_daily_data["Reported_Cases"]
for i in range(len(data)):
    if i + window_size < len(data):
        moving_average_cases.append(round(np.mean(data[i:i+window_size]),1))
    else:
        moving_average_cases.append(np.mean(data[i:len(data)]))

moving_average_deaths = []
data = covid_daily_data["death_cases"]
for i in range(len(data)):
    if i + window_size < len(data):
        moving_average_deaths.append(round(np.mean(data[i:i+window_size]),1))
    else:
        moving_average_deaths.append(np.mean(data[i:len(data)]))


,Date,Reported_Cases,death_cases
85,2023-01-22,3,0
86,2023-01-23,1,0
87,2023-01-24,6,0
88,2023-01-25,13,0
89,2023-01-26,5,0
